## imports

In [203]:
from multiprocessing import Event, Pool, Queue
import time

import cv2
import numpy as np

import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger
from cflib.positioning.position_hl_commander import PositionHlCommander
from cflib.positioning.motion_commander import MotionCommander

## constants

In [204]:
group_number = 4
uri = f'radio://0/{group_number}/2M'
camera = 2
delay_ms = 0
frame_buf_maxsize = 5
cmd_buf_maxsize = 10
buf_timeout = 1

## cf helper functions

In [205]:
def wait_for_position_estimator(scf):
    print('Waiting for estimator to find position...')

    log_config = LogConfig(name='Kalman Variance', period_in_ms=500)
    log_config.add_variable('kalman.varPX', 'float')
    log_config.add_variable('kalman.varPY', 'float')
    log_config.add_variable('kalman.varPZ', 'float')

    var_y_history = [1000] * 10
    var_x_history = [1000] * 10
    var_z_history = [1000] * 10

    threshold = 0.001
    with SyncLogger(scf, log_config) as logger:
        for log_entry in logger:
            data = log_entry[1]

            var_x_history.append(data['kalman.varPX'])
            var_x_history.pop(0)
            var_y_history.append(data['kalman.varPY'])
            var_y_history.pop(0)
            var_z_history.append(data['kalman.varPZ'])
            var_z_history.pop(0)

            min_x = min(var_x_history)
            max_x = max(var_x_history)
            min_y = min(var_y_history)
            max_y = max(var_y_history)
            min_z = min(var_z_history)
            max_z = max(var_z_history)

            print("{} {} {}".
                format(max_x - min_x, max_y - min_y, max_z - min_z))

            if (max_x - min_x) < threshold and (
                    max_y - min_y) < threshold and (
                    max_z - min_z) < threshold:
                break

def set_PID_controller(cf):
    # Set the PID Controller:
    print('Initializing PID Controller')
    cf.param.set_value('stabilizer.controller', '1')
    cf.param.set_value('kalman.resetEstimation', '1')
    time.sleep(0.1)
    cf.param.set_value('kalman.resetEstimation', '0')
    
    wait_for_position_estimator(cf)
    time.sleep(0.1)
    return

## init

In [206]:
def init_cf():
    # Initialize all the CrazyFlie drivers:
    cflib.crtp.init_drivers(enable_debug_driver=False)

    # Scan for Crazyflies in range of the antenna:
    print('Scanning interfaces for Crazyflies...')
    available = cflib.crtp.scan_interfaces()

    # List local CrazyFlie devices:
    print('Crazyflies found:')
    for i in available:
        print(i[0])
        
    if len(available) == 0:
        try:
            raise Exception('No Crazyflies found, cannot run example')
        except:
            return None
        
    scf = SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache'))
    
    set_PID_controller(scf.cf)
    
    return scf.cf

def init(f_b, c_b, st):
    global frame_buf, cmd_buf, stop, cf, start_time, failed
    frame_buf = f_b
    cmd_buf = c_b
    stop = st
#     cf = init_cf()
    cf = 1
    start_time = time.perf_counter_ns()
    failed = False
    
    if cf is None:
        failed = True

## ascend

In [ ]:
def ascend_and_hover():
    if failed:
        return
    
    # Ascend:
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, y / 10)
        time.sleep(0.1)
    
    # Hover at 1 meter:
    for _ in range(20):
        cf.commander.send_hover_setpoint(0, 0, 0, 1)
        time.sleep(0.1)

## capture

In [207]:
def capture_frames(camera, delay_ms):
    if failed:
        return
    
    cap = cv2.VideoCapture(camera)
    cap.set(cv2.CAP_PROP_BUFFERSIZE, 2)
    
    while cap.isOpened() and not stop.is_set():
        ok, frame = cap.read()

        if ok:
            # TODO: make sure queue doesnt fill up
            frame_buf.put_nowait(frame)
        else:
            break

        time.sleep(delay_ms / 1000)
            
    cap.release()
    frame_buf.close()

## planning

In [208]:
def process_frame(buf_timeout):
    if failed:
        return
    
    while not stop.is_set():
        try:
            frame = frame_buf.get(timeout=buf_timeout)
        except Exception as e:
            break

        cmd_buf.put_nowait((1, 1, 1, 0))
        
    cmd_buf.close()

## command

In [209]:
def send_command(buf_timeout):
    if failed:
        return
    
    while not stop.is_set():
        try:
            x, y, z, yaw = cmd_buf.get(timeout=buf_timeout)
        except Exception as e:
            break

#         cf.commander.send_position_setpoint(x, y, z, yaw)

## descend

In [ ]:
def hover_and_descend():
    if failed:
        return
    
    # Hover at 1 meter:
    for _ in range(30):
        cf.commander.send_hover_setpoint(0, 0, 0, 1)
        time.sleep(0.1)
    
    # Descend:
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, (10 - y) / 10)
        time.sleep(0.1)
    
    # Stop all motion:
    for i in range(10):
        cf.commander.send_stop_setpoint()
        time.sleep(0.1)

## main loop

In [210]:
# TEMPORARY
def f():
    stop.set()

In [211]:
frame_buf = Queue(frame_buf_maxsize)
cmd_buf = Queue(cmd_buf_maxsize)
stop = Event()

pool = Pool(6, initializer=init, initargs=(frame_buf, cmd_buf, stop))

ascend_future = pool.apply(ascend_and_hover)

capture_frames_future = pool.apply_async(capture_frames, args=(camera, delay_ms))
process_frame_future = pool.apply_async(process_frame, args=(buf_timeout,))
send_command_future = pool.apply_async(send_command, args=(buf_timeout,))

descend_future = pool.apply(hover_and_descend)

time.sleep(5)
f_future = pool.apply_async(f)

pool.close()
pool.join()

# TODO

* check cf init
* close things before descent
* actually think about detection
* keep track of position?
* figure out how to trigger descent